In [181]:
import pymongo
import re
import nltk

def load_data(database_name, collection_name):
    # Connect to MongoDB
    client = pymongo.MongoClient("mongodb://localhost:27017/")
    db = client[database_name]
    collection = db[collection_name]

    # Load data from MongoDB
    data = list(collection.find())

    return data


 # Load data from MongoDB
database_name = "lab1"
collection_name = "diseases"
raw_data = load_data(database_name, collection_name)

In [62]:
raw_data[0]

{'_id': ObjectId('66137a12432dd74b0b4d7428'),
 'count': 1,
 'illness_type': 'داء السكري',
 'disease_name': 'التغذية الصحية لمرضى السكري المصابين بأمراض الكلى',
 'content': '<div id="ctl00_PlaceHolderMain_ctl02__ControlWrapper_RichHtmlField" class="ms-rtestate-field" style="display:inline" aria-labelledby="ctl00_PlaceHolderMain_ctl02_label"><p style="text-align:justify;">\u200b<strong>الخطة الغذائية لمرضى الكلى:</strong></p><p style="text-align:justify;">المكملات العشبية ليست آمنة في حالة مرضى الكلى، كما أن بعض الفيتامينات يمكن أن تضر مرضى الكلى أيضًا، لذلك يجب استشارة الطبيب قبل تناول أي نوع منها، وتعتمد الخطة الغذائية لمريض الكلى على حالة المريض وما إذا كان في مرحلة متقدمة أو متأخرة من المرض؛ وعليها يُوصَى بالحد من، أو تجنُّب بعض الأطعمة الغذائية، ويوصى لمريض الكلى بالآتي:<br></p><ul style="text-align:justify;"><li>الحد من الصوديوم: حيث مع مُضِي الوقت يفقد مريض الكلى تدريجيًّا القدرة على تحقيق التوازن بين المياه والصوديوم بالجسم؛ لذلك يساعد الحد من الصوديوم على خفض ضغط الدم وتقليل احت

In [186]:
# I changed the '' into ' ' to assure that no words get accidentally stuck together.
# removed the \u200b one
# found three articles with a different structure
# encoding was done throughly by using write for json, and then to mongoDB, using compass to visualize
# got the scraping by layers so that I would make sure to get good results
# made it exclude all the punctuations manually except for the shadda

import re

def clean_data(data):
    cleaned_data = []
    for entry in data:
        cleaned_entry = {}

        # Clean 'illness_type'
        if 'illness_type' in entry and entry['illness_type']:
            illness_type = re.sub(r'\u200b', ' ', entry['illness_type'])
            cleaned_entry['illness_type'] = illness_type.strip()
        else:
            cleaned_entry['illness_type'] = ''

        # Clean 'disease_name'
        if 'disease_name' in entry and entry['disease_name']:
            disease_name = re.sub(r'\u200b', ' ', entry['disease_name'])
            cleaned_entry['disease_name'] = disease_name.strip()
        else:
            cleaned_entry['disease_name'] = ''

        # Clean 'content'
        if 'content' in entry and entry['content']:
            # Remove HTML tags and '\u200b'
            content = re.sub(r'<[^>]+>|[\u200b]', ' ', entry['content'])
           
            # Remove extra whitespaces
            content = re.sub(r'\s+', ' ', content).strip()

            # Remove parentheses
            content = content.replace('(', ' ').replace(')', ' ')
            
            cleaned_entry['content'] = content
        else:
            cleaned_entry['content'] = ''

        # Add cleaned entry to the list
        cleaned_data.append(cleaned_entry)

    return cleaned_data

# Example usage
cleaned_data = clean_data(raw_data)


In [188]:
cleaned_data[2]

{'illness_type': 'أمراض القلب والدورة الدموية',
 'disease_name': 'انخفاض ضغط الدم الموضعي',
 'content': 'انخفاض ضغط الدم الوضعي: هو حالة ينخفض فيها ضغط الدم، مما يسبب انخفاض مستوى تدفق الدم إلى المخ بشكل كبير عند الوقوف بشكل سريع ومفاجئ، كما قد يؤدي إلى الدوخة والسقوط والإغماء وقد تحدث الأعراض عند: الوقوف المفاجئ بعد الجلوس. الجلوس المفاجئ بعد الاستلقاء. الاستيقاظ في الصباح. بعد تناول وجبة كبيرة. خلال ممارسة الرياضة. الشعور بالقلق أو الذعر. السبب: مرض السكري، فشل القلب أو تصلب الشرايين. تناول بعض مدرات البول، مضادات الاكتئاب أو الأدوية لخفض ضغط الدم. الأمراض العصبية  مثل: مرض باركنسون وبعض أنواع الخرف . الجفاف. نقص فيتامين ب 12، أو فقر الدم. إدمان الكحول. الراحة المطولة في السرير. الأعراض: يحدث انخفاض ضغط الدم الانتصابي بشكل متكرر مع أعراض أكثر حدة في الصباح؛ وذلك لأن ضغط الدم يكون عند أدنى مستوياته في الصباح، يمكن أن تختلف من شخص لآخر، وقد تشمل: الدوار عند الوقوف. الشعور بالإغماء أو السقوط. الصداع وعدم وضوح الرؤية. الشعور بالضغط عبر الجزء الخلفي من الكتف أو العنق. الشعور بالغثيان أو ب

In [ ]:
nltk.download('punkt')
from nltk.tokenize import word_tokenize

In [189]:
#Sentences Tokenizing for POS and NER
def tokenize_sentences(data):
    tokenized_data = []
    for entry in data:
        content = entry.get('content')
        if content:
            sentences = sent_tokenize(content)
            entry_without_content = {key: value for key, value in entry.items() if key != 'content'}
            entry_without_content['sentences'] = sentences
            tokenized_data.append(entry_without_content)
    return tokenized_data

# Example usage:
tokenized_sentences_data = tokenize_sentences(cleaned_data)
tokenized_sentences_data[0]

{'illness_type': 'داء السكري',
 'disease_name': 'التغذية الصحية لمرضى السكري المصابين بأمراض الكلى',
 'sentences': ['الخطة الغذائية لمرضى الكلى: المكملات العشبية ليست آمنة في حالة مرضى الكلى، كما أن بعض الفيتامينات يمكن أن تضر مرضى الكلى أيضًا، لذلك يجب استشارة الطبيب قبل تناول أي نوع منها، وتعتمد الخطة الغذائية لمريض الكلى على حالة المريض وما إذا كان في مرحلة متقدمة أو متأخرة من المرض؛ وعليها يُوصَى بالحد من، أو تجنُّب بعض الأطعمة الغذائية، ويوصى لمريض الكلى بالآتي: الحد من الصوديوم: حيث مع مُضِي الوقت يفقد مريض الكلى تدريجيًّا القدرة على تحقيق التوازن بين المياه والصوديوم بالجسم؛ لذلك يساعد الحد من الصوديوم على خفض ضغط الدم وتقليل احتباس السوائل بالجسم وهو أمر شائع مع مرضى الكلى.',
  'التركيز على الطعام الطازج والمطهو بالمنزل، وتناول كميات صغيرة من طعام المطاعم والأطعمة المعلبة لأنها غالبًا ما تحتوي على الكثير من الصوديوم.',
  'اختيار منتجات الغذاء التي تحتوي على نسبة صوديوم أقل من 5% على ملصقها الغذائي.',
  'استبدال الملح بتعزيز النكهات بالأعشاب والتوابل والخردل والخل وفي غضون أسبوع

In [191]:
def clean_data_final(data):
    cleaned_data = []
    for entry in data:
        if 'content' in entry and entry['content']:
            # Extract content from the entry
            content = entry['content']

            # Remove specific punctuations and symbols except for the shadda and percent sign
            content = re.sub(r'[\.,;:_\*\-\[\]\(\)!؟<>،؛،»«\{\}"\'/\\—]', ' ', content)
            
            # Add cleaned content to the entry
            cleaned_entry = {
                'illness_type': entry['illness_type'],
                'disease_name': entry['disease_name'],
                'content': content
            }

            cleaned_data.append(cleaned_entry)
        else:
            print("Skipping entry with missing or empty content:", entry)

    return cleaned_data

# Example usage
cleaned_data_final = clean_data_final(cleaned_data)
cleaned_data_final[2]

{'illness_type': 'أمراض القلب والدورة الدموية',
 'disease_name': 'انخفاض ضغط الدم الموضعي',
 'content': 'انخفاض ضغط الدم الوضعي  هو حالة ينخفض فيها ضغط الدم  مما يسبب انخفاض مستوى تدفق الدم إلى المخ بشكل كبير عند الوقوف بشكل سريع ومفاجئ  كما قد يؤدي إلى الدوخة والسقوط والإغماء وقد تحدث الأعراض عند  الوقوف المفاجئ بعد الجلوس  الجلوس المفاجئ بعد الاستلقاء  الاستيقاظ في الصباح  بعد تناول وجبة كبيرة  خلال ممارسة الرياضة  الشعور بالقلق أو الذعر  السبب  مرض السكري  فشل القلب أو تصلب الشرايين  تناول بعض مدرات البول  مضادات الاكتئاب أو الأدوية لخفض ضغط الدم  الأمراض العصبية  مثل  مرض باركنسون وبعض أنواع الخرف   الجفاف  نقص فيتامين ب 12  أو فقر الدم  إدمان الكحول  الراحة المطولة في السرير  الأعراض  يحدث انخفاض ضغط الدم الانتصابي بشكل متكرر مع أعراض أكثر حدة في الصباح  وذلك لأن ضغط الدم يكون عند أدنى مستوياته في الصباح  يمكن أن تختلف من شخص لآخر  وقد تشمل  الدوار عند الوقوف  الشعور بالإغماء أو السقوط  الصداع وعدم وضوح الرؤية  الشعور بالضغط عبر الجزء الخلفي من الكتف أو العنق  الشعور بالغثيان أو ب

In [192]:
#Word Tokenizing
def tokenize_sentences(data):
    tokenized_data = []
    for entry in data:
        content = entry.get('content')
        if content:
            words = word_tokenize(content)
            entry_without_content = {key: value for key, value in entry.items() if key != 'content'}
            entry_without_content['words'] = words
            tokenized_data.append(entry_without_content)
    return tokenized_data

# Example usage:
tokenized_data_words = tokenize_sentences(cleaned_data_final)
tokenized_data_words[0]

{'illness_type': 'داء السكري',
 'disease_name': 'التغذية الصحية لمرضى السكري المصابين بأمراض الكلى',
 'words': ['الخطة',
  'الغذائية',
  'لمرضى',
  'الكلى',
  'المكملات',
  'العشبية',
  'ليست',
  'آمنة',
  'في',
  'حالة',
  'مرضى',
  'الكلى',
  'كما',
  'أن',
  'بعض',
  'الفيتامينات',
  'يمكن',
  'أن',
  'تضر',
  'مرضى',
  'الكلى',
  'أيضًا',
  'لذلك',
  'يجب',
  'استشارة',
  'الطبيب',
  'قبل',
  'تناول',
  'أي',
  'نوع',
  'منها',
  'وتعتمد',
  'الخطة',
  'الغذائية',
  'لمريض',
  'الكلى',
  'على',
  'حالة',
  'المريض',
  'وما',
  'إذا',
  'كان',
  'في',
  'مرحلة',
  'متقدمة',
  'أو',
  'متأخرة',
  'من',
  'المرض',
  'وعليها',
  'يُوصَى',
  'بالحد',
  'من',
  'أو',
  'تجنُّب',
  'بعض',
  'الأطعمة',
  'الغذائية',
  'ويوصى',
  'لمريض',
  'الكلى',
  'بالآتي',
  'الحد',
  'من',
  'الصوديوم',
  'حيث',
  'مع',
  'مُضِي',
  'الوقت',
  'يفقد',
  'مريض',
  'الكلى',
  'تدريجيًّا',
  'القدرة',
  'على',
  'تحقيق',
  'التوازن',
  'بين',
  'المياه',
  'والصوديوم',
  'بالجسم',
  'لذلك',
  'يساعد',
  

In [136]:
nltk.download('stopwords')
from nltk.corpus import stopwords

print(stopwords.words('arabic'))

['إذ', 'إذا', 'إذما', 'إذن', 'أف', 'أقل', 'أكثر', 'ألا', 'إلا', 'التي', 'الذي', 'الذين', 'اللاتي', 'اللائي', 'اللتان', 'اللتيا', 'اللتين', 'اللذان', 'اللذين', 'اللواتي', 'إلى', 'إليك', 'إليكم', 'إليكما', 'إليكن', 'أم', 'أما', 'أما', 'إما', 'أن', 'إن', 'إنا', 'أنا', 'أنت', 'أنتم', 'أنتما', 'أنتن', 'إنما', 'إنه', 'أنى', 'أنى', 'آه', 'آها', 'أو', 'أولاء', 'أولئك', 'أوه', 'آي', 'أي', 'أيها', 'إي', 'أين', 'أين', 'أينما', 'إيه', 'بخ', 'بس', 'بعد', 'بعض', 'بك', 'بكم', 'بكم', 'بكما', 'بكن', 'بل', 'بلى', 'بما', 'بماذا', 'بمن', 'بنا', 'به', 'بها', 'بهم', 'بهما', 'بهن', 'بي', 'بين', 'بيد', 'تلك', 'تلكم', 'تلكما', 'ته', 'تي', 'تين', 'تينك', 'ثم', 'ثمة', 'حاشا', 'حبذا', 'حتى', 'حيث', 'حيثما', 'حين', 'خلا', 'دون', 'ذا', 'ذات', 'ذاك', 'ذان', 'ذانك', 'ذلك', 'ذلكم', 'ذلكما', 'ذلكن', 'ذه', 'ذو', 'ذوا', 'ذواتا', 'ذواتي', 'ذي', 'ذين', 'ذينك', 'ريث', 'سوف', 'سوى', 'شتان', 'عدا', 'عسى', 'عل', 'على', 'عليك', 'عليه', 'عما', 'عن', 'عند', 'غير', 'فإذا', 'فإن', 'فلا', 'فمن', 'في', 'فيم', 'فيما', 'فيه', 'فيها', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [193]:
def apply_stopwords(tokenized_data):
    stop_words = set(stopwords.words('arabic'))  # Arabic stopwords
    stopwords_data = []
    for entry in tokenized_data:
        words_without_stopwords = [word for word in entry['words'] if word not in stop_words]
        entry_without_stopwords = {
            'illness_type': entry['illness_type'],
            'disease_name': entry['disease_name'],
            'words': words_without_stopwords
        }
        stopwords_data.append(entry_without_stopwords)
    return stopwords_data

# Example usage:
stopwords_data = apply_stopwords(tokenized_data_words)
stopwords_data[0]


{'illness_type': 'داء السكري',
 'disease_name': 'التغذية الصحية لمرضى السكري المصابين بأمراض الكلى',
 'words': ['الخطة',
  'الغذائية',
  'لمرضى',
  'الكلى',
  'المكملات',
  'العشبية',
  'آمنة',
  'حالة',
  'مرضى',
  'الكلى',
  'الفيتامينات',
  'يمكن',
  'تضر',
  'مرضى',
  'الكلى',
  'أيضًا',
  'لذلك',
  'يجب',
  'استشارة',
  'الطبيب',
  'تناول',
  'نوع',
  'وتعتمد',
  'الخطة',
  'الغذائية',
  'لمريض',
  'الكلى',
  'حالة',
  'المريض',
  'مرحلة',
  'متقدمة',
  'متأخرة',
  'المرض',
  'وعليها',
  'يُوصَى',
  'بالحد',
  'تجنُّب',
  'الأطعمة',
  'الغذائية',
  'ويوصى',
  'لمريض',
  'الكلى',
  'بالآتي',
  'الحد',
  'الصوديوم',
  'مُضِي',
  'الوقت',
  'يفقد',
  'مريض',
  'الكلى',
  'تدريجيًّا',
  'القدرة',
  'تحقيق',
  'التوازن',
  'المياه',
  'والصوديوم',
  'بالجسم',
  'لذلك',
  'يساعد',
  'الحد',
  'الصوديوم',
  'خفض',
  'ضغط',
  'الدم',
  'وتقليل',
  'احتباس',
  'السوائل',
  'بالجسم',
  'أمر',
  'شائع',
  'مرضى',
  'الكلى',
  'التركيز',
  'الطعام',
  'الطازج',
  'والمطهو',
  'بالمنزل',
  'وت

In [194]:
import nltk
from nltk.stem import SnowballStemmer

# Download the SnowballStemmer resource for Arabic
nltk.download('snowball_data')

# Function to apply stemming to the words in the data
def apply_arabic_stemming(data):
    stemmer = SnowballStemmer('arabic')
    stemmed_data = []
    for entry in data:
        if 'words' in entry and entry['words']:
            # Apply stemming to each word in the list
            stemmed_words = [stemmer.stem(word) for word in entry['words']]
            # Update the 'words' key with the stemmed words
            entry['words'] = stemmed_words
            # Append the updated entry to the stemmed data list
            stemmed_data.append(entry)
    return stemmed_data

# Apply Arabic stemming to the tokenized data
stemmed_data = apply_arabic_stemming(stopwords_data)
stemmed_data[0]

[nltk_data] Downloading package snowball_data to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package snowball_data is already up-to-date!


{'illness_type': 'داء السكري',
 'disease_name': 'التغذية الصحية لمرضى السكري المصابين بأمراض الكلى',
 'words': ['الخط',
  'غذاء',
  'مرضى',
  'كلى',
  'مكمل',
  'عشب',
  'امن',
  'حال',
  'مرضى',
  'كلى',
  'فيتامين',
  'يمك',
  'تضر',
  'مرضى',
  'كلى',
  'ايض',
  'لذل',
  'يجب',
  'استشار',
  'طبيب',
  'تناول',
  'نوع',
  'تعتمد',
  'الخط',
  'غذاء',
  'مريض',
  'كلى',
  'حال',
  'مريض',
  'مرحل',
  'متقدم',
  'متاخر',
  'مرض',
  'وعل',
  'يوصى',
  'الحد',
  'تجنب',
  'اطعم',
  'غذاء',
  'يوصى',
  'مريض',
  'كلى',
  'الات',
  'الحد',
  'صوديوم',
  'مض',
  'الو',
  'يفقد',
  'مريض',
  'كلى',
  'تدريج',
  'قدر',
  'تحقيق',
  'تواز',
  'مياه',
  'صوديوم',
  'جسم',
  'لذل',
  'يساعد',
  'الحد',
  'صوديوم',
  'خفض',
  'ضغط',
  'الدم',
  'تقليل',
  'احتباس',
  'سوايل',
  'جسم',
  'امر',
  'شايع',
  'مرضى',
  'كلى',
  'تركيز',
  'طعام',
  'طازج',
  'والمط',
  'منزل',
  'تناول',
  'كم',
  'صغير',
  'طعام',
  'مطاعم',
  'اطعم',
  'معلب',
  'لان',
  'غالب',
  'تحتو',
  'كثير',
  'صوديوم',
  'ا

In [195]:
import re
import nltk
from nltk.stem import WordNetLemmatizer

# Download necessary resources for Arabic lemmatization
nltk.download('wordnet')

def lemmatize_arabic(data):
    lemmatizer = WordNetLemmatizer()
    
    for entry in data:
        if 'words' in entry and entry['words']:
            # Lemmatize Arabic words
            words = entry['words']
            lemmatized_words = [lemmatizer.lemmatize(word, pos='v') for word in words]  # Assuming verbs for Arabic
            
            # Update entry with lemmatized words
            entry['lemmatized_words'] = lemmatized_words
        else:
            print("Skipping entry with missing or empty words:", entry)

    return data

# Example usage
lemmatized_data = lemmatize_arabic(stopwords_data)
lemmatized_data[0]

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


{'illness_type': 'داء السكري',
 'disease_name': 'التغذية الصحية لمرضى السكري المصابين بأمراض الكلى',
 'words': ['الخط',
  'غذاء',
  'مرضى',
  'كلى',
  'مكمل',
  'عشب',
  'امن',
  'حال',
  'مرضى',
  'كلى',
  'فيتامين',
  'يمك',
  'تضر',
  'مرضى',
  'كلى',
  'ايض',
  'لذل',
  'يجب',
  'استشار',
  'طبيب',
  'تناول',
  'نوع',
  'تعتمد',
  'الخط',
  'غذاء',
  'مريض',
  'كلى',
  'حال',
  'مريض',
  'مرحل',
  'متقدم',
  'متاخر',
  'مرض',
  'وعل',
  'يوصى',
  'الحد',
  'تجنب',
  'اطعم',
  'غذاء',
  'يوصى',
  'مريض',
  'كلى',
  'الات',
  'الحد',
  'صوديوم',
  'مض',
  'الو',
  'يفقد',
  'مريض',
  'كلى',
  'تدريج',
  'قدر',
  'تحقيق',
  'تواز',
  'مياه',
  'صوديوم',
  'جسم',
  'لذل',
  'يساعد',
  'الحد',
  'صوديوم',
  'خفض',
  'ضغط',
  'الدم',
  'تقليل',
  'احتباس',
  'سوايل',
  'جسم',
  'امر',
  'شايع',
  'مرضى',
  'كلى',
  'تركيز',
  'طعام',
  'طازج',
  'والمط',
  'منزل',
  'تناول',
  'كم',
  'صغير',
  'طعام',
  'مطاعم',
  'اطعم',
  'معلب',
  'لان',
  'غالب',
  'تحتو',
  'كثير',
  'صوديوم',
  'ا

In [196]:
import re

# Removing diacritics
def remove_diacritics(word_or_list):
    # Define a regular expression pattern to match Arabic diacritics
    pattern = r'[\u064B-\u065F\u0670]'  # Range of Arabic diacritical marks
    
    # If input is a list, apply remove_diacritics recursively to each element
    if isinstance(word_or_list, list):
        return [remove_diacritics(word) for word in word_or_list]
    
    # If input is a string, remove diacritics using the pattern
    elif isinstance(word_or_list, str):
        return re.sub(pattern, '', word_or_list)
    
    # Otherwise, return the input unchanged
    else:
        return word_or_list

# Apply remove_diacritics function to stopwords_data
cleaned_stopwords_data = remove_diacritics(stopwords_data)

# Print words without diacritics
for i, words in enumerate(cleaned_stopwords_data[:100]):  # Print the first few phrases
    print(f"Phrase {i+1} words (without diacritics):", words)

Phrase 1 words (without diacritics): {'illness_type': 'داء السكري', 'disease_name': 'التغذية الصحية لمرضى السكري المصابين بأمراض الكلى', 'words': ['الخط', 'غذاء', 'مرضى', 'كلى', 'مكمل', 'عشب', 'امن', 'حال', 'مرضى', 'كلى', 'فيتامين', 'يمك', 'تضر', 'مرضى', 'كلى', 'ايض', 'لذل', 'يجب', 'استشار', 'طبيب', 'تناول', 'نوع', 'تعتمد', 'الخط', 'غذاء', 'مريض', 'كلى', 'حال', 'مريض', 'مرحل', 'متقدم', 'متاخر', 'مرض', 'وعل', 'يوصى', 'الحد', 'تجنب', 'اطعم', 'غذاء', 'يوصى', 'مريض', 'كلى', 'الات', 'الحد', 'صوديوم', 'مض', 'الو', 'يفقد', 'مريض', 'كلى', 'تدريج', 'قدر', 'تحقيق', 'تواز', 'مياه', 'صوديوم', 'جسم', 'لذل', 'يساعد', 'الحد', 'صوديوم', 'خفض', 'ضغط', 'الدم', 'تقليل', 'احتباس', 'سوايل', 'جسم', 'امر', 'شايع', 'مرضى', 'كلى', 'تركيز', 'طعام', 'طازج', 'والمط', 'منزل', 'تناول', 'كم', 'صغير', 'طعام', 'مطاعم', 'اطعم', 'معلب', 'لان', 'غالب', 'تحتو', 'كثير', 'صوديوم', 'اختيار', 'منتج', 'غذاء', 'تحتو', 'نسب', 'صوديوم', '5', '%', 'ملصق', 'غذاء', 'استبدال', 'ملح', 'تعزيز', 'نكه', 'اعشاب', 'توابل', 'خردل', 'والخل',

In [197]:
import unicodedata

def normalize_words(words):
    normalized_words = []
    for word in words:
        # Check if the word is a list (nested list)
        if isinstance(word, list):
            # If it's a nested list, recursively call normalize_words
            normalized_sublist = normalize_words(word)
            normalized_words.append(normalized_sublist)
        else:
            # Unicode normalization (NFKD form)
            normalized_word = unicodedata.normalize('NFKD', word)
        
            # Case normalization (convert to lowercase)
            normalized_word = normalized_word.lower()
        
            # Whitespace normalization (replace multiple spaces with a single space)
            normalized_word = ' '.join(normalized_word.split())
        
            normalized_words.append(normalized_word)
    
    return normalized_words

def normalize_stopwords_data(stopwords_data):
    normalized_stopwords_data = []
    for entry in stopwords_data:
        # Normalize the words in each entry
        normalized_entry = {
            'illness_type': entry['illness_type'],
            'disease_name': entry['disease_name'],
            'words': normalize_words(entry['words'])
        }
        normalized_stopwords_data.append(normalized_entry)
    return normalized_stopwords_data

# Example usage
normalized_stopwords_data = normalize_stopwords_data(stopwords_data)
print(normalized_stopwords_data[0])

{'illness_type': 'داء السكري', 'disease_name': 'التغذية الصحية لمرضى السكري المصابين بأمراض الكلى', 'words': ['الخط', 'غذاء', 'مرضى', 'كلى', 'مكمل', 'عشب', 'امن', 'حال', 'مرضى', 'كلى', 'فيتامين', 'يمك', 'تضر', 'مرضى', 'كلى', 'ايض', 'لذل', 'يجب', 'استشار', 'طبيب', 'تناول', 'نوع', 'تعتمد', 'الخط', 'غذاء', 'مريض', 'كلى', 'حال', 'مريض', 'مرحل', 'متقدم', 'متاخر', 'مرض', 'وعل', 'يوصى', 'الحد', 'تجنب', 'اطعم', 'غذاء', 'يوصى', 'مريض', 'كلى', 'الات', 'الحد', 'صوديوم', 'مض', 'الو', 'يفقد', 'مريض', 'كلى', 'تدريج', 'قدر', 'تحقيق', 'تواز', 'مياه', 'صوديوم', 'جسم', 'لذل', 'يساعد', 'الحد', 'صوديوم', 'خفض', 'ضغط', 'الدم', 'تقليل', 'احتباس', 'سوايل', 'جسم', 'امر', 'شايع', 'مرضى', 'كلى', 'تركيز', 'طعام', 'طازج', 'والمط', 'منزل', 'تناول', 'كم', 'صغير', 'طعام', 'مطاعم', 'اطعم', 'معلب', 'لان', 'غالب', 'تحتو', 'كثير', 'صوديوم', 'اختيار', 'منتج', 'غذاء', 'تحتو', 'نسب', 'صوديوم', '5', '%', 'ملصق', 'غذاء', 'استبدال', 'ملح', 'تعزيز', 'نكه', 'اعشاب', 'توابل', 'خردل', 'والخل', 'وف', 'غضو', 'اسبوع', 'اسبوع', 'سيعت

In [ ]:
import stanza

# Download the Arabic language model for Stanza
stanza.download('ar')

# Initialize the Arabic Stanza pipeline
nlp = stanza.Pipeline('ar', processors='tokenize, pos')

def pos_tag_sentences(tokenized_sentences_data):
    pos_tagged_data = []
    for entry in tokenized_sentences_data:
        sentences = entry.get('sentences', [])
        pos_tags_sentences = []
        for sentence in sentences:
            # Perform POS tagging using Stanza
            doc = nlp(sentence)
            pos_tags = [(word.text, word.upos) for sent in doc.sentences for word in sent.words]
            pos_tags_sentences.append(pos_tags)
        pos_tagged_entry = {
            'illness_type': entry.get('illness_type', ''),
            'disease_name': entry.get('disease_name', ''),
            'pos_tags_sentences': pos_tags_sentences
        }
        pos_tagged_data.append(pos_tagged_entry)
    return pos_tagged_data

# Example usage:
pos_tagged_data = pos_tag_sentences(tokenized_sentences_data)
pos_tagged_data[0]


In [ ]:
import stanza

# Download the Arabic language model for Stanza
stanza.download('ar')

# Initialize the Arabic Stanza pipeline for NER
nlp_ner = stanza.Pipeline('ar', processors='tokenize, ner')

def ner_tag_words(stopwords_data):
    ner_tagged_data = []
    for entry in stopwords_data:
        words = entry.get('words', [])
        ner_tags = []
        for word in words:
            # Perform NER tagging using Stanza
            doc = nlp_ner(word)
            ner_tag = doc.entities
            ner_tags.append((word, ner_tag))
        ner_tagged_entry = {
            'illness_type': entry.get('illness_type', ''),
            'disease_name': entry.get('disease_name', ''),
            'ner_tags': ner_tags
        }
        ner_tagged_data.append(ner_tagged_entry)
    return ner_tagged_data

# Example usage:
ner_tagged_data = ner_tag_words(stopwords_data)
ner_tagged_data[0]


2024-04-08 10:50:48 INFO: Downloaded file to C:\Users\HP\stanza_resources\resources.json
2024-04-08 10:50:48 INFO: Downloading default packages for language: ar (Arabic) ...
2024-04-08 10:50:50 INFO: File exists: C:\Users\HP\stanza_resources\ar\default.zip
2024-04-08 10:50:55 INFO: Finished downloading models and saved to C:\Users\HP\stanza_resources
2024-04-08 10:50:55 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-04-08 10:50:56 INFO: Downloaded file to C:\Users\HP\stanza_resources\resources.json
2024-04-08 10:50:56 WARNING: Language ar package default expects mwt, which has been added
2024-04-08 10:50:57 INFO: Loading these models for language: ar (Arabic):
| Processor | Package      |
----------------------------
| tokenize  | padt         |
| mwt       | padt         |
| ner       | aqmar_charlm |

2024-04-08 10:50:57 INFO: Using device: cpu
2024-04-08 10:50:57 INFO: Loading: tokenize
2024-04-08 10:50:57 INFO: Loading: mwt
2024-04-08 10:50:57 INFO: Loading: ner
2024-04-08 10:50:58 INFO: Done loading processors!
